In [ ]:
import os
from time import gmtime, strftime

#### Defined by User

In [ ]:
## specify your Dkube username
DKUBEUSERNAME = "ocdkube"

## Define the model monitor name here that you will be creating 
MONITOR_NAME = "image-mm-sgmkr-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
## Specify the data source

PREFIX = "sagemaker/Demo-ImageModelMonitor"

## Dkube information
TOKEN = os.getenv("DKUBE_USER_ACCESS_TOKEN","")
DKUBE_URL = os.getenv("DKUBE_URL","")
DKUBE_TRAINING_CODE_NAME = "monitoring-examples"

### Specify Sagemaker details
BUCKET = os.getenv("BUCKET","")
ROLE = os.getenv("ROLE","")
REGION_NAME = os.getenv("REGION_NAME","")

ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID","")
SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY","")


### Dkube cluster 
SAGEMAKER_DKUBE_CLUSTER_NAME = "aws-sagemaker-cluster"

# dataset to be used as training data
TRAINING_DATASET = 'chest-xray'

# the frequency with which monitoring will run
RUN_FREQUENCY = 5

LIVE_DATASET = MONITOR_NAME+'-aws'

if TOKEN == '' or DKUBEUSERNAME == '' or DKUBE_URL == '':
    raise TypeError("Please fill the Dkube details first (TOKEN, DKUBE_URL, DKUBEUSERNAME)")
if ACCESS_KEY=='' or SECRET_KEY=='' or ROLE=='' or REGION_NAME=='' or BUCKET=='':
    raise TypeError("Please fill the AWS_S3 details first (ACCESS_KEY, SECRET_KEY, ROLE, REGION_NAME, BUCKET)")

In [ ]:
image_sgmkr_config = {"MONITOR_NAME":MONITOR_NAME, "DKUBEUSERNAME":DKUBEUSERNAME,
                      "TOKEN":TOKEN, "DKUBE_URL":DKUBE_URL, "ACCESS_KEY":ACCESS_KEY,
                      "SECRET_KEY":SECRET_KEY, "TRAINING_DATASET":TRAINING_DATASET,
                      "LIVE_DATASET":LIVE_DATASET,
                      "RUN_FREQUENCY":RUN_FREQUENCY, "DKUBE_TRAINING_CODE_NAME":DKUBE_TRAINING_CODE_NAME,
                      "BUCKET":BUCKET, "PREFIX":PREFIX, "ROLE":ROLE, "REGION_NAME":REGION_NAME,
                      "SAGEMAKER_DKUBE_CLUSTER_NAME":SAGEMAKER_DKUBE_CLUSTER_NAME}
%store image_sgmkr_config

#### Dkube Resources

In [ ]:
import time,json,shutil
from dkube.sdk import *

In [ ]:
api = DkubeApi(URL=DKUBE_URL,token=TOKEN)
if DKUBEUSERNAME == api.validate_token()['username']:
    pass
else:
    print("Invalid User, please check your username, first")

#### Dataset

In [ ]:
try:
    dataset = DkubeDataset(DKUBEUSERNAME, name=TRAINING_DATASET)
    dataset.update_dataset_source(source="git")
    dataset.update_git_details(url="https://github.com/oneconvergence/dkube-examples/tree/monitoring/image_cloudevents/data/chest-xray-mini")
    api.create_dataset(dataset)
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")

### Model Monitor Datasets

##### Labelled Dataset

In [ ]:
try:
    dataset = DkubeDataset(DKUBEUSERNAME, name=LIVE_DATASET,remote=True)
    dataset.update_dataset_source('aws_s3')
    dataset.update_awss3_details(bucket=BUCKET,prefix='',
                                key=os.getenv("AWS_ACCESS_KEY_ID",ACCESS_KEY),
                                secret=os.getenv("AWS_SECRET_ACCESS_KEY",SECRET_KEY))
    api.create_dataset(dataset)
except Exception as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
    else:
        raise e

#### Cleanup

In [ ]:
## Set CLEANUP = True, after your experiment is complete.
CLEANUP = False
if CLEANUP:
    api.delete_dataset(DKUBEUSERNAME,TRAINING_DATASET,force=True)
    api.delete_dataset(DKUBEUSERNAME,LIVE_DATASET)
    %store -d image_sgmkr_config